# Tratamiento de datos para obtener una muestra de votantes

El estudio va a utilizar la muestra del ENEMDU del 2020, a diciembre, para simular la decisión del voto de la primera vuelta.
La intención es usar datos de la misma encuensta a abril del 2021, para simular los de la segunda vueta de las elecciones del 2021, pero de momento no existen los identificadores de hogar en la base del INEC respectiva, como para hacerlo.

In [178]:
# Importar paquetes 

import pandas as pd
import pyreadstat as pyd

In [179]:
# Crear dfs para los módulos 

df_persona, meta_p = pyd.read_sav('Data/Enemdu_2020/enemdu_persona_2020_12.sav', apply_value_formats=True)
df_consumidor, meta_c = pyd.read_sav('Data/Enemdu_2020/enemdu_consumidor_2020_12.sav', apply_value_formats=True)
df_vivienda, meta_v = pyd.read_sav('Data/Enemdu_2020/enemdu_vivienda_hogar_2020_12.sav', apply_value_formats=True)

In [180]:
# Vamos a decodificar el nombre de las variables

df_persona.rename(columns=meta_p.column_names_to_labels, inplace=True) # meta.column_names_to_labels sirve como diccionarios
df_consumidor.rename(columns=meta_c.column_names_to_labels, inplace=True)
df_vivienda.rename(columns=meta_v.column_names_to_labels, inplace=True)

# Tipología del hogar, según INEC

In [181]:
for i in df_persona.columns:
    print(i)

Area
Ciudad
Conglomerado
Panelm
Vivienda
Hogar
Persona
Sexo
Edad
Relación de parentesco
Seguro Social - Alternativa 1
Seguro Social - Alternativa 2
Estado civil
Asiste a clases
Jornada a la que asiste
Modalidad de estudio a la que asiste
Tipo de conexión para actividades educativas
Razón por la que no asiste
Nivel de instrucción
Año aprobado
Sabe leer y escribir
Por sus estudios obtuvo algún título
Qué título obtuvo
Cómo se considera
Donde nació
Lugar de nacimiento
Código del Informante
Trabajó la semana pasada
Actividad que realizó para ayudar en su hogar
Tiene trabajo del cual estuvo ausente
Razón por la que no trabajó
Razón relacionada con el COVID19
Continua(ó) recibiendo algún ingreso durante ausencia del trabajo
Desde que dejó de trabajar hasta que espera regresar ¿Cuánto tiempo estará (...) ausente
Horas de trabajo en la semana anterior
Razón por la que trabajó menos de 40 horas
Razón relacionada con el COVID19
Razón por la que trabajó más de 40 horas
Desea trabajar más horas
Es

In [182]:
# Ver los posibles resultados de la Relación de parentesco

df_persona['Relación de parentesco'].values

['Jefe', 'Hijo o Hija', 'Jefe', 'Cónyuge', 'Hijo o Hija', ..., 'Hijo o Hija', 'Hijo o Hija', 'Hijo o Hija', 'Hijo o Hija', 'Yerno o nuera']
Length: 30646
Categories (9, object): ['Cónyuge', 'Empleado(a) doméstico(a)', 'Hijo o Hija', 'Jefe', ..., 'Otros no parientes', 'Otros parientes', 'Padres o suegros', 'Yerno o nuera']

# Número de personas según relación de parentesco

In [183]:
c_par = df_persona.groupby(by=['Identificador de hogar', 
                               'Relación de parentesco']).size().reset_index(name='cuenta_parentesco')
c_par
# cuenta_parentesco es la variable que muestra el número de persona para cada relación de parentesco

,Identificador de hogar,Relación de parentesco,cuenta_parentesco
0,010150000503009011,Cónyuge,1
1,010150000503009011,Empleado(a) doméstico(a),0
2,010150000503009011,Hijo o Hija,1
3,010150000503009011,Jefe,1
4,010150000503009011,Nieto o nieta,0
...,...,...,...
78799,240350001406005071,Nieto o nieta,1
78800,240350001406005071,Otros no parientes,0
78801,240350001406005071,Otros parientes,0
78802,240350001406005071,Padres o suegros,0


## Número de personas en el hogar que son parientes y por género

Hay que tener un cuenta que ya no es posible distinguir a parientes filiales de los no fialiales, porque por ejemplo una persona del hogar puede responder que es padre (persona filial) o suegro (no filiar) del jefe de hogar. 

In [184]:
c_par = c_par.pivot(index='Identificador de hogar', columns='Relación de parentesco', values='cuenta_parentesco')
c_par = c_par.reset_index()
c_par['Parientes'] = c_par['Nieto o nieta'] + c_par['Otros parientes'] + c_par['Padres o suegros'] + c_par['Yerno o nuera'] # Parientes no incluye hijos
c_par['No parientes'] = c_par['Cónyuge'] + c_par['Otros no parientes']
c_par = c_par[['Identificador de hogar', 'Cónyuge', 'Hijo o Hija', 'Parientes', 'No parientes', 'Empleado(a) doméstico(a)']]
# Para más información revisar https://pandas.pydata.org/pandas-docs/stable/user_guide/reshaping.html

In [185]:
c_par

Relación de parentesco,Identificador de hogar,Cónyuge,Hijo o Hija,Parientes,No parientes,Empleado(a) doméstico(a)
0,010150000503009011,1,1,0,1,0
1,010150000503009021,0,4,3,0,0
2,010150000503009031,0,4,3,0,0
3,010150000503009041,1,1,0,1,0
4,010150000503009051,1,2,0,1,0
...,...,...,...,...,...,...
8751,240350001406005031,0,1,2,0,0
8752,240350001406005041,1,4,0,1,0
8753,240350001406005051,0,1,2,0,0
8754,240350001406005061,1,3,0,1,0


In [186]:
# Necesitamos una variable más para proceder a tipificar el hogar ecuatoriano, el género del jefe del hogar. Está codificada 
# como Sexo. 

gender = df_persona[df_persona['Relación de parentesco'] == 'Jefe'][['Identificador de hogar', 'Sexo']]
c_par = c_par.merge(gender, on=['Identificador de hogar'])
# Cambiar el orden de las columnas
c_par = c_par.rename(columns={'Sexo': 'Género del jefe/a de hogar'})
c_par['Género del jefe/a de hogar'] = c_par['Género del jefe/a de hogar'].str.slice(start=1) # La variable tiene un espacio innecesario
c_par = c_par.iloc[:, [0, 6, 1, 2, 3, 4, 5]]
c_par

,Identificador de hogar,Género del jefe/a de hogar,Cónyuge,Hijo o Hija,Parientes,No parientes,Empleado(a) doméstico(a)
0,010150000503009011,Hombre,1,1,0,1,0
1,010150000503009021,Mujer,0,4,3,0,0
2,010150000503009031,Mujer,0,4,3,0,0
3,010150000503009041,Hombre,1,1,0,1,0
4,010150000503009051,Hombre,1,2,0,1,0
...,...,...,...,...,...,...,...
8751,240350001406005031,Mujer,0,1,2,0,0
8752,240350001406005041,Hombre,1,4,0,1,0
8753,240350001406005051,Mujer,0,1,2,0,0
8754,240350001406005061,Hombre,1,3,0,1,0


# Algorítmo de tipificación 

In [187]:
# Los nans son diferentes de 0, así que usamos un pequeños truco para cerciorarnos, ya que si el valor fuera na
# significaría que ese tipo de variable no existe.

v = ['Cónyuge', 'Hijo o Hija', 'Parientes', 'No parientes']
for variable in v:
    c_par[variable] = c_par[variable].fillna(0)


# Agregamos una columna para reemplazar valor

T = []

for i in range(c_par.shape[0]):
    if c_par['Cónyuge'][i] != 0:
        if c_par['Hijo o Hija'][i] != 0:
            if c_par['Parientes'][i] != 0:
                if c_par['No parientes'][i] != 0:
                    T.append('Hogar tipo 16')
                else:
                    T.append('Hogar tipo 6')
            elif c_par['No parientes'][i] != 0:
                T.append('Hogar tipo 10')
            else:
                T.append('Hogar tipo 2')
        elif c_par['Parientes'][i] != 0:
            if c_par['No parientes'][i] != 0:
                T.append('Hogar tipo 16')
            else:
                T.append('Hogar tipo 7')
        elif c_par['No parientes'][i] != 0:
            T.append('Hogar tipo 11')
        else:
            T.append('Hogar tipo 3')
    else:
        if c_par['Hijo o Hija'][i] != 0:
            if c_par['Parientes'][i] != 0:
                if c_par['No parientes'][i] != 0:
                    T.append('Hogar tipo 16')
                else:
                    if c_par['Género del jefe/a de hogar'][i] == 'Hombre':
                        T.append('Hogar tipo 8')
                    elif c_par['Género del jefe/a de hogar'][i] == 'Mujer':
                        T.append('Hogar tipo 9')
                    else:
                        T.append('nan')
            elif c_par['No parientes'][i] != 0:
                if c_par['Género del jefe/a de hogar'][i] == 'Hombre':
                    T.append('Hogar tipo 12')
                elif c_par['Género del jefe/a de hogar'][i] == 'Mujer':
                    T.append('Hogar tipo 13')
                else:
                    T.append('nan')
            else:
                if c_par['Género del jefe/a de hogar'][i] == 'Hombre':
                    T.append('Hogar tipo 4')
                elif c_par['Género del jefe/a de hogar'][i] == 'Mujer':
                    T.append('Hogar tipo 5')
                else:
                    T.append('nan')
        elif c_par['Parientes'][i] != 0:
            if c_par['No parientes'][i] != 0:
                T.append('Hogar tipo 16')
            else:
                T.append('Hogar tipo 14')
        elif c_par['No parientes'][i] != 0:
            T.append('Hogar tipo 15')
        else:
            T.append('Hogar tipo 1')


# Agregar columna en la base de datos

c_par['Tipology'] = T


# c_par.to_csv('C:/Users/micha/Downloads/YAF.csv', index=False)

In [188]:
c_par

,Identificador de hogar,Género del jefe/a de hogar,Cónyuge,Hijo o Hija,Parientes,No parientes,Empleado(a) doméstico(a),Tipology
0,010150000503009011,Hombre,1,1,0,1,0,Hogar tipo 10
1,010150000503009021,Mujer,0,4,3,0,0,Hogar tipo 9
2,010150000503009031,Mujer,0,4,3,0,0,Hogar tipo 9
3,010150000503009041,Hombre,1,1,0,1,0,Hogar tipo 10
4,010150000503009051,Hombre,1,2,0,1,0,Hogar tipo 10
...,...,...,...,...,...,...,...,...
8751,240350001406005031,Mujer,0,1,2,0,0,Hogar tipo 9
8752,240350001406005041,Hombre,1,4,0,1,0,Hogar tipo 10
8753,240350001406005051,Mujer,0,1,2,0,0,Hogar tipo 9
8754,240350001406005061,Hombre,1,3,0,1,0,Hogar tipo 10


## Verificación de consistencia

Ahora hay que verificar que todos los hogares hayan sido tipificados. Vamos a empezar creando un descriptivo del número de hogares por código de tipo.

In [189]:
table = c_par.groupby(by=['Tipology']).size().reset_index(name='N')
table.sort_values('Tipology')

,Tipology,N
0,Hogar tipo 1,1161
1,Hogar tipo 10,2914
2,Hogar tipo 11,929
3,Hogar tipo 12,6
4,Hogar tipo 13,6
5,Hogar tipo 14,355
6,Hogar tipo 15,31
7,Hogar tipo 16,1476
8,Hogar tipo 4,201
9,Hogar tipo 5,1009


Vamos a confirmalo verificando que el número de hogares de la base ajustada y la base bruta sean los mismo a continuación.

In [190]:
a = c_par.groupby(by=['Tipology']).size().sum()
b = c_par.shape[0]
print('La suma del número de hogares es igual a la cantidad de hogares de la base bruta del ENEMDU: ' + str(a) + '=' + str(b))

La suma del número de hogares es igual a la cantidad de hogares de la base bruta del ENEMDU: 8756=8756


Para ser más exhaustivos, vamos a verificar que del hogar tipo 2 que no fue tipificado no exista ni un solo caso en la base. Esto se hace filtrando la base de datos por condicionales, lo que sería equivalente a identificar todos los hogares  de tipo 2 en la base. 

In [191]:
c_par.loc[((c_par['Cónyuge'] != 0) & (c_par['Hijo o Hija'] != 0) & (c_par['Parientes'] == 0) & (c_par['No parientes'] == 0)), ]

,Identificador de hogar,Género del jefe/a de hogar,Cónyuge,Hijo o Hija,Parientes,No parientes,Empleado(a) doméstico(a),Tipology


Para cerciorarnos más vamos a verificar dentro de la base si existen hogares sin parientes y no parientes donde el jefe de hogar tenga cónyuge.

In [192]:
c_par.loc[((c_par['No parientes'] == 0) & (c_par['Parientes'] == 0) & (c_par['Cónyuge'] == 1)), ]

,Identificador de hogar,Género del jefe/a de hogar,Cónyuge,Hijo o Hija,Parientes,No parientes,Empleado(a) doméstico(a),Tipology


Ahora podemos estar seguros de que los cálculos implementados son correctos, pero también podemos dudar sobre la representatividad de la muestra proporcionada por el INEC.

## Mapa interactivo según tipo de hogar 

In [193]:
import numpy as np
import geopandas as gpd
import folium 

In [194]:
# Agregamos los códigos de ciudad en la base c_par

c_par = c_par.merge(df_vivienda[['Ciudad', 'Identificador de hogar']], on='Identificador de hogar')
c_par['Ciudad'] = c_par['Ciudad'].astype(int).astype(str)
c_par.head(4)

,Identificador de hogar,Género del jefe/a de hogar,Cónyuge,Hijo o Hija,Parientes,No parientes,Empleado(a) doméstico(a),Tipology,Ciudad
0,010150000503009011,Hombre,1,1,0,1,0,Hogar tipo 10,10150
1,010150000503009021,Mujer,0,4,3,0,0,Hogar tipo 9,10150
2,010150000503009031,Mujer,0,4,3,0,0,Hogar tipo 9,10150
3,010150000503009041,Hombre,1,1,0,1,0,Hogar tipo 10,10150


In [195]:
# Crear una tabla que contenga el número de jefes o jefas del hogar del muestreo para la ciudad respectiva

j_gender = c_par.groupby(by=['Ciudad', 'Género del jefe/a de hogar']).size().reset_index(name='Número de jefes/as de hoagr por género')
j_gender = j_gender.pivot(index='Ciudad', columns='Género del jefe/a de hogar', values='Número de jefes/as de hoagr por género').reset_index()
j_gender = j_gender.rename(columns={'Hombre': 'Número de jefes de hogar', 'Mujer': 'Número de jefas de hogar'})
j_gender

Género del jefe/a de hogar,Ciudad,Número de jefes de hogar,Número de jefas de hogar
0,100150,4.00,3.00
1,100152,6.00,1.00
2,100153,4.00,3.00
3,100157,7.00,7.00
4,100250,3.00,4.00
...,...,...,...
361,92150,5.00,1.00
362,92250,6.00,1.00
363,92251,10.00,4.00
364,92350,9.00,5.00


In [196]:
# Contar el número de hogares por ciudad. (Aquí es más útil usar la función count).

N = c_par.groupby(by=['Ciudad'])['Identificador de hogar'].count().reset_index(name='Número de hogares en la ciudad encuestada')
N

,Ciudad,Número de hogares en la ciudad encuestada
0,100150,7
1,100152,7
2,100153,7
3,100157,14
4,100250,7
...,...,...
361,92150,6
362,92250,7
363,92251,14
364,92350,14


In [197]:
# Número de hogares

# Crear una tabla que contenga el número de jefes o jefas del hogar del muestreo para la ciudad respectiva

N_tipo = c_par.groupby(by=['Ciudad', 'Tipology']).size().reset_index(name='Número de hogares por tipo')
N_tipo = N_tipo.pivot(index='Ciudad', columns='Tipology', values='Número de hogares por tipo').reset_index()
N_tipo = N_tipo.merge(N, on='Ciudad')
N_tipo

,Ciudad,Hogar tipo 1,Hogar tipo 10,Hogar tipo 11,Hogar tipo 12,Hogar tipo 13,Hogar tipo 14,Hogar tipo 15,Hogar tipo 16,Hogar tipo 4,Hogar tipo 5,Hogar tipo 8,Hogar tipo 9,Número de hogares en la ciudad encuestada
0,100150,2.00,3.00,NaN,NaN,NaN,NaN,NaN,1.00,NaN,1.00,NaN,NaN,7
1,100152,1.00,2.00,3.00,NaN,NaN,NaN,NaN,1.00,NaN,NaN,NaN,NaN,7
2,100153,1.00,NaN,NaN,NaN,NaN,1.00,NaN,2.00,NaN,2.00,NaN,1.00,7
3,100157,1.00,4.00,NaN,NaN,NaN,NaN,NaN,2.00,1.00,5.00,NaN,1.00,14
4,100250,2.00,1.00,NaN,NaN,NaN,NaN,NaN,3.00,NaN,1.00,NaN,NaN,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
361,92150,1.00,1.00,NaN,NaN,NaN,NaN,NaN,3.00,NaN,NaN,NaN,1.00,6
362,92250,1.00,3.00,2.00,NaN,NaN,NaN,NaN,1.00,NaN,NaN,NaN,NaN,7
363,92251,3.00,2.00,3.00,NaN,NaN,NaN,NaN,2.00,1.00,2.00,NaN,1.00,14
364,92350,3.00,2.00,2.00,NaN,NaN,1.00,NaN,2.00,NaN,3.00,1.00,NaN,14


In [198]:
# Load maps

mapa = gpd.read_file('Data/mapa_ecuador/', layer='ecu_admbnda_adm3_inec_20190724')
mapa['Ciudad'] = mapa['ADM3_PCODE'].str.slice(start=2).astype(int).astype(str)
mapa.head(6)

,ADM3_ES,ADM3_PCODE,ADM3_REF,ADM3ALT1ES,ADM3ALT2ES,ADM2_ES,ADM2_PCODE,ADM1_ES,ADM1_PCODE,ADM0_EN,ADM0_ES,ADM0_PCODE,date,validOn,validTo,Shape_Leng,Shape_Area,geometry,Ciudad
0,10 de Agosto,EC131752,None,None,None,Pedernales,EC1317,Manabi,EC13,Ecuador,Ecuador (el),EC,2018-12-17,2019-07-24,None,0.65,0.02,"POLYGON ((-80.06370 -0.04398, -80.06400 -0.044...",131752
1,10 de Agosto,EC210160,None,None,None,Lago Agrio,EC2101,Sucumbios,EC21,Ecuador,Ecuador (el),EC,2018-12-17,2019-07-24,None,0.42,0.00,"POLYGON ((-76.89351 0.24284, -76.88845 0.22824...",210160
2,11 de Noviembre (Ilinchisi),EC050157,None,None,None,Latacunga,EC0501,Cotopaxi,EC05,Ecuador,Ecuador (el),EC,2018-12-17,2019-07-24,None,0.12,0.00,"POLYGON ((-78.65780 -0.89633, -78.65733 -0.897...",50157
3,12 de Diciembre (Cab. en Achiotes),EC111452,None,None,None,Pindal,EC1114,Loja,EC11,Ecuador,Ecuador (el),EC,2018-12-17,2019-07-24,None,0.40,0.00,"POLYGON ((-80.13218 -4.02787, -80.13193 -4.028...",111452
4,16 de Agosto,EC140455,None,None,None,Palora,EC1404,Morona Santiago,EC14,Ecuador,Ecuador (el),EC,2018-12-17,2019-07-24,None,0.55,0.01,"POLYGON ((-77.90145 -1.68901, -77.90020 -1.689...",140455
5,27 de Abril (Cab. en la Naranja),EC110654,None,None,None,Espindola,EC1106,Loja,EC11,Ecuador,Ecuador (el),EC,2018-12-17,2019-07-24,None,0.31,0.00,"POLYGON ((-79.45788 -4.41708, -79.45780 -4.417...",110654


## Mapa según número de hogares de tipo

In [199]:
# Unimos las bases

mapa = mapa.merge(N_tipo, on=['Ciudad'])
mapa.columns

Index(['ADM3_ES', 'ADM3_PCODE', 'ADM3_REF', 'ADM3ALT1ES', 'ADM3ALT2ES',
       'ADM2_ES', 'ADM2_PCODE', 'ADM1_ES', 'ADM1_PCODE', 'ADM0_EN', 'ADM0_ES',
       'ADM0_PCODE', 'date', 'validOn', 'validTo', 'Shape_Leng', 'Shape_Area',
       'geometry', 'Ciudad', 'Hogar tipo 1', 'Hogar tipo 10', 'Hogar tipo 11',
       'Hogar tipo 12', 'Hogar tipo 13', 'Hogar tipo 14', 'Hogar tipo 15',
       'Hogar tipo 16', 'Hogar tipo 4', 'Hogar tipo 5', 'Hogar tipo 8',
       'Hogar tipo 9', 'Número de hogares en la ciudad encuestada'],
      dtype='object')

In [200]:
# Me quedo con las variables necesarias

N_tipo = mapa[['Ciudad', 'ADM3_ES', 'Shape_Leng', 'Shape_Area',
               'geometry', 'Hogar tipo 1', 'Hogar tipo 10', 'Hogar tipo 11',
               'Hogar tipo 12', 'Hogar tipo 13', 'Hogar tipo 14', 'Hogar tipo 15',
               'Hogar tipo 16', 'Hogar tipo 4', 'Hogar tipo 5', 'Hogar tipo 8',
               'Hogar tipo 9', 'Número de hogares en la ciudad encuestada']]

In [201]:
Mapa_N_tipo = N_tipo.explore(column='Número de hogares en la ciudad encuestada',  # make choropleth based on "BoroName" column
                             scheme="naturalbreaks",  # use mapclassify's natural breaks scheme
                             legend=True, # show legend
                             k=10, # use 10 bins
                             legend_kwds=dict(colorbar=False), # do not use colorbar
                             name="ADM3_ES" # name of the layer in the map
                            )

Mapa_N_tipo.save('Maps\hogares.html')

# Selección de la muestra 

El número de personas en el hogar no incluye a la empleada o empleoado doméstico. Incialmente, se había pensado incluirla, pero existen asmitreías de conexión y roles para que se puede una persona que tiene un efecto en lso otros miembros del hogar. Por ejemplo, la intención de la variable número de mujeres es medir si existe alguna tendencia de voto más progresista en el hogar, por la influencia de discursos feministas cuya probabilidad de presencia aumenta con el número de mujeres de cierto nivel de instrucción.

In [202]:
# Nota: Personas que no son empleados/as domésticas. 

# Número de personas del hogar
a = df_persona.loc[df_persona['Relación de parentesco'] != 'Empleado(a) doméstico(a)', ].groupby(['Identificador de hogar'])['Identificador de persona'].size().reset_index(name='Número de personas del hogar')
# Número de mujeres del hogar
b = df_persona.loc[( ( df_persona['Relación de parentesco'] != 'Empleado(a) doméstico(a)' ) & ( df_persona['Sexo'] == ' Mujer' ) ), ].groupby(['Identificador de hogar']).size().reset_index(name='Número de mujeres en el hogar')
# Junte de a y b
N_m = b.merge(a, on='Identificador de hogar')
Muestra = c_par.merge(N_m, on=['Identificador de hogar'])
Muestra.head(10)

,Identificador de hogar,Género del jefe/a de hogar,Cónyuge,Hijo o Hija,Parientes,No parientes,Empleado(a) doméstico(a),Tipology,Ciudad,Número de mujeres en el hogar,Número de personas del hogar
0,010150000503009011,Hombre,1,1,0,1,0,Hogar tipo 10,10150,2,3
1,010150000503009021,Mujer,0,4,3,0,0,Hogar tipo 9,10150,4,8
2,010150000503009031,Mujer,0,4,3,0,0,Hogar tipo 9,10150,6,8
3,010150000503009041,Hombre,1,1,0,1,0,Hogar tipo 10,10150,1,3
4,010150000503009051,Hombre,1,2,0,1,0,Hogar tipo 10,10150,2,4
5,010150000503009081,Hombre,1,2,0,1,0,Hogar tipo 10,10150,2,4
6,010150001101005011,Hombre,1,0,1,1,0,Hogar tipo 16,10150,1,3
7,010150001101005021,Hombre,1,2,0,1,0,Hogar tipo 10,10150,3,4
8,010150001101005031,Mujer,0,3,0,0,0,Hogar tipo 5,10150,2,4
9,010150001101005041,Hombre,1,1,0,1,0,Hogar tipo 10,10150,2,3


# Selección de variables, módulo persona

In [203]:
[f for f in df_persona.columns]

['Area',
 'Ciudad',
 'Conglomerado',
 'Panelm',
 'Vivienda',
 'Hogar',
 'Persona',
 'Sexo',
 'Edad',
 'Relación de parentesco',
 'Seguro Social - Alternativa 1',
 'Seguro Social - Alternativa 2',
 'Estado civil',
 'Asiste a clases',
 'Jornada a la que asiste',
 'Modalidad de estudio a la que asiste',
 'Tipo de conexión para actividades educativas',
 'Razón por la que no asiste',
 'Nivel de instrucción',
 'Año aprobado',
 'Sabe leer y escribir',
 'Por sus estudios obtuvo algún título',
 'Qué título obtuvo',
 'Cómo se considera',
 'Donde nació',
 'Lugar de nacimiento',
 'Código del Informante',
 'Trabajó la semana pasada',
 'Actividad que realizó para ayudar en su hogar',
 'Tiene trabajo del cual estuvo ausente',
 'Razón por la que no trabajó',
 'Razón relacionada con el COVID19',
 'Continua(ó) recibiendo algún ingreso durante ausencia del trabajo',
 'Desde que dejó de trabajar hasta que espera regresar ¿Cuánto tiempo estará (...) ausente',
 'Horas de trabajo en la semana anterior',
 'Ra

In [204]:
['Area', 
 'Ciudad', 
 'Sexo', 
 'Edad', 
 'Seguro Social - Alternativa 1', # Ver si es seguro social del jefe
 'Seguro Social - Alternativa 2', # Ver si es seguro social de la cónyuge
 'Estado civil', 
 'Nivel de instrucción',
 'Cómo se considera',
 'Trabajó la semana pasada', # ** Trabajo --
 'Continua(ó) recibiendo algún ingreso durante ausencia del trabajo'
 'Semanas que busca trabajo',
 'Desea trabajar y está dispuesto a hacerlo',
 'Condición de inactividad', 
 'Rama de actividad',
 'Grupo de ocupación',
 'Categoría de ocupación', 
 'Recibe alimentación',
 'Recibe vivienda',
 'Recibe transporte',
 'Recibe vacaciones',
 'Recibe ropa de trabajo',
 'Recibe seguro social',
 'Recibe seguro médico',
 'Recibe guardería',
 'Recibe cursos de capacitación',
 'Recibe 13er sueldo',
 'Recibe 14to sueldo',
 'Cuántos años trabaja',
 'Condición de actividad',
 'Ingreso Laboral', 
 'Grupo de Ocupación CIUO8 (población ocupada de 15 años y más)',
 'Rama de Actividad CIIU4 (población ocupada de 15 años y más)',
 'Ingreso per cápita',
 'Pobreza',
 'Extrema Pobreza',
 'Nivel de instrucción',
 'Identificador de persona'] 

['Area',
 'Ciudad',
 'Sexo',
 'Edad',
 'Seguro Social - Alternativa 1',
 'Seguro Social - Alternativa 2',
 'Estado civil',
 'Nivel de instrucción',
 'Cómo se considera',
 'Trabajó la semana pasada',
 'Continua(ó) recibiendo algún ingreso durante ausencia del trabajoSemanas que busca trabajo',
 'Desea trabajar y está dispuesto a hacerlo',
 'Condición de inactividad',
 'Rama de actividad',
 'Grupo de ocupación',
 'Categoría de ocupación',
 'Recibe alimentación',
 'Recibe vivienda',
 'Recibe transporte',
 'Recibe vacaciones',
 'Recibe ropa de trabajo',
 'Recibe seguro social',
 'Recibe seguro médico',
 'Recibe guardería',
 'Recibe cursos de capacitación',
 'Recibe 13er sueldo',
 'Recibe 14to sueldo',
 'Cuántos años trabaja',
 'Condición de actividad',
 'Ingreso Laboral',
 'Grupo de Ocupación CIUO8 (población ocupada de 15 años y más)',
 'Rama de Actividad CIIU4 (población ocupada de 15 años y más)',
 'Ingreso per cápita',
 'Pobreza',
 'Extrema Pobreza',
 'Nivel de instrucción',
 'Identifi

'Cómo se considera' es etnia (mestizo, mulato, etc).

## Lista a tener en cuenta
Estas variables aparentan la mayor relevancia:

In [205]:
persona_variables = ['Identificador de persona',
                     'Identificador de hogar',
                     'Area', 
                     'Ciudad', 
                     'Sexo',
                     'Edad',
                     'Seguro Social - Alternativa 1', # Ver si es seguro social del jefe
                     'Seguro Social - Alternativa 2', # Ver si es seguro social de la cónyuge
                     'Estado civil', 
                     'Nivel de instrucción',
                     'Cómo se considera',
                     'Condición de inactividad', 
                     'Categoría de ocupación',
                     'Cuántos años trabaja',
                     'Condición de actividad',
                     'Ingreso Laboral',
                     'Grupo de Ocupación CIUO8 (población ocupada de 15 años y más)',
                     'Rama de Actividad CIIU4 (población ocupada de 15 años y más)',
                     'Ingreso per cápita', 
                     'Pobreza',
                     'Extrema Pobreza'] 

# El ingreso per cápita por el número de miembros del hogar es el ingreso familar

In [206]:
# Hay dos variables de nombre Nivel de instrucción y vamos a eleminar la primera, porque la segunda contiene
# mejor detalle. La columna nivel de instrucción es la 9, y es la que vamos a eliminar:

df_persona = df_persona.drop([9])
persona_variables = df_persona[persona_variables]

Ahora, vamos a seleccionar variables de interés de los módulos consumidor y vivienda.

In [207]:
# Módulo vivienda

variables_vivienda = df_vivienda[['Identificador de hogar', 'Forma de tenencia de la vivienda']] 

In [208]:
# Módulo consumo

variables_consumo = ['Con relación al mes anterior, la situación económica de su hogar es:',
                     'Cómo cree usted que será la situación económica de su hogar dentro de los próximos 3 meses:',
                     'Planea usted o algún miembro del hogar adquirir un vehículo en los próximos 3 meses',
                     'Planea usted o algún miembro del hogar adquirir un inmueble (casa, departamento, pieza) en los próximos 3 meses',
                     'Usted o algún miembro del hogar tiene planes de endeudamiento en los próximos 3 meses (bancos, financieras, etc.)',
                     'Con relación al mes anterior, la situación económica del país es:',
                     'Usted considera que durante los próximos 3 meses la situación económica del país será:',
                     'En los 3 meses anteriores, considera que la situación en Ecuador para encontrar/mejorar un puesto de trabajo es:',
                     'Considera usted que la situación en Ecuador para encontrar/mejorar un puesto de trabajo dentro de 3 meses será:'
                     ] 


variables_consumo = df_consumidor[['Identificador de hogar']+variables_consumo]
variables_consumo


,Identificador de hogar,"Con relación al mes anterior, la situación económica de su hogar es:",Cómo cree usted que será la situación económica de su hogar dentro de los próximos 3 meses:,Planea usted o algún miembro del hogar adquirir un vehículo en los próximos 3 meses,"Planea usted o algún miembro del hogar adquirir un inmueble (casa, departamento, pieza) en los próximos 3 meses","Usted o algún miembro del hogar tiene planes de endeudamiento en los próximos 3 meses (bancos, financieras, etc.)","Con relación al mes anterior, la situación económica del país es:",Usted considera que durante los próximos 3 meses la situación económica del país será:,"En los 3 meses anteriores, considera que la situación en Ecuador para encontrar/mejorar un puesto de trabajo es:",Considera usted que la situación en Ecuador para encontrar/mejorar un puesto de trabajo dentro de 3 meses será:
0,010150000503009011,Igual,Igual,No,No,No,Igual,Igual,Igual,Igual
1,010150000503009021,Peor,Igual,No,No,No,Peor,Peor,Peor,Peor
2,010150000503009031,Peor,Mejor,No,No,No,Peor,Igual,Peor,Peor
3,010150000503009041,Peor,Mejor,No,No,No,Igual,Igual,Peor,Peor
4,010150000503009051,Igual,Igual,No,No,No,Peor,Peor,Peor,Peor
...,...,...,...,...,...,...,...,...,...,...
8751,240350001406005031,Igual,Peor,No,No,No,Peor,Peor,Peor,Peor
8752,240350001406005041,Peor,Peor,No,No,No,Peor,Peor,Peor,Peor
8753,240350001406005051,Igual,Igual,No,No,No,Igual,Igual,Peor,Igual
8754,240350001406005061,Igual,Igual,No,No,No,Peor,Peor,Peor,Peor


### Muestra para el estudio *¿Por quién votan los ecuatorianos y por qué? Factores sociales, económicos y demográficos.*

In [209]:
persona_variables = persona_variables.merge(variables_vivienda, on=['Identificador de hogar'])
persona_variables = persona_variables.merge(variables_consumo, on=['Identificador de hogar'])

# Ya que los votantes son mayores de 16 años debido al voto facultativo, vamos a filtrar la base para
# poder identificar a los potenciales votantes.

In [210]:
# Hay un problema que resolver antes: la variable edad está codificada como categoría.

persona_variables['Edad'].replace({' 98 y más': '99'}, inplace=True)

C:\Users\micha\Documents\Electoral\Electoral\lib\site-packages\pandas\core\generic.py:6619: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)


In [211]:
# Volvemos la variable float

persona_variables['Edad'] = persona_variables['Edad'].astype(float)

In [212]:
# Finalmete filtramos la base de datos

persona_variables = persona_variables[persona_variables['Edad'] >= 16]

In [213]:
persona_variables = persona_variables.merge(Muestra, on='Identificador de hogar')

In [214]:
persona_variables = persona_variables.drop(columns=['Ciudad_x'])
persona_variables.rename(columns={'Ciudad_y': 'Ciudad'}, inplace=True)

In [215]:
persona_variables

,Identificador de persona,Identificador de hogar,Area,Sexo,Edad,Seguro Social - Alternativa 1,Seguro Social - Alternativa 2,Estado civil,Nivel de instrucción,Nivel de instrucción,...,Género del jefe/a de hogar,Cónyuge,Hijo o Hija,Parientes,No parientes,Empleado(a) doméstico(a),Tipology,Ciudad,Número de mujeres en el hogar,Número de personas del hogar
0,20025000002200502101,200250000022005021,Rural,Mujer,55.00,Ninguno,Ninguno,Separado(a),Secundaria,Educación Básica,...,Mujer,0,1,0,0,0,Hogar tipo 5,200250,1,2
1,20025000002200502102,200250000022005021,Rural,Hombre,39.00,Ninguno,Ninguno,Soltero(a),Secundaria,Educación Básica,...,Mujer,0,1,0,0,0,Hogar tipo 5,200250,1,2
2,20025000002200503101,200250000022005031,Rural,Mujer,30.00,"IESS, seguro general voluntario",Ninguno,Casado(a),Superior Universitario,Superior,...,Mujer,1,1,0,1,0,Hogar tipo 10,200250,2,3
3,20025000002200503102,200250000022005031,Rural,Hombre,33.00,Ninguno,Ninguno,Casado(a),Superior Universitario,Superior,...,Mujer,1,1,0,1,0,Hogar tipo 10,200250,2,3
4,20025000002200505101,200250000022005051,Rural,Mujer,58.00,Ninguno,Ninguno,Divorciado(a),Secundaria,Educación Media/Bachillerato,...,Mujer,0,1,1,0,0,Hogar tipo 9,200250,1,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21828,05015091010200909101,050150910102009091,Rural,Hombre,48.00,Ninguno,Ninguno,Casado(a),Primaria,Educación Básica,...,Hombre,1,4,1,1,0,Hogar tipo 16,50150,3,7
21829,05015091010200909102,050150910102009091,Rural,Mujer,45.00,Ninguno,Ninguno,Casado(a),Primaria,Educación Básica,...,Hombre,1,4,1,1,0,Hogar tipo 16,50150,3,7
21830,05015091010200909103,050150910102009091,Rural,Hombre,21.00,Ninguno,Ninguno,Soltero(a),Educación Media,Educación Media/Bachillerato,...,Hombre,1,4,1,1,0,Hogar tipo 16,50150,3,7
21831,05015091010200909104,050150910102009091,Rural,Hombre,17.00,Ninguno,Ninguno,Unión libre,Educación Media,Educación Media/Bachillerato,...,Hombre,1,4,1,1,0,Hogar tipo 16,50150,3,7


In [218]:
# Mostar 2 decimales

pd.options.display.float_format = '{:,.2f}'.format
persona_variables.to_csv('Data/muestra.csv', index_label=False, encoding='utf-8')